In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from clean import CleanData
import seaborn as sns
import json
from googletrans import Translator


In [4]:
df = pd.read_csv("new_dfm.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               1000 non-null   int64  
 1   Unnamed: 0.1             1000 non-null   int64  
 2   id                       1000 non-null   int64  
 3   text                     962 non-null    object 
 4   createTime               1000 non-null   object 
 5   webVideoUrl              1000 non-null   object 
 6   likeCount                1000 non-null   int64  
 7   shareCount               1000 non-null   int64  
 8   playCount                1000 non-null   int64  
 9   commentCount             1000 non-null   int64  
 10  downloaded               1000 non-null   bool   
 11  mentions                 1000 non-null   object 
 12  hashtags                 1000 non-null   object 
 13  authorMeta.id            1000 non-null   int64  
 14  authorMeta.secUid        

In [6]:
df["translated_lang"].value_counts().head(20)


original sound                                   353
Original sound                                    27
Original sound.                                   20
as original                                       19
Therefore I Am                                     8
Original                                           6
Oh No                                              5
The original sound                                 5
Potential Breakup Song                             4
WITHOUT YOU                                        4
Where Is The Love?                                 4
Original Sound                                     4
What You Know Bout Love                            3
They are original                                  3
Pilfingerdansen - Remix                            3
ruin our friendship                                3
positions                                          2
BILLIE JEAN X BOO X F IT UP VMESHBEATS MASHUP      2
Rocky: Eye of the Tiger                       

In [7]:
df["translated_lang"] = df["translated_lang"].astype(
    str)
df["translated_lang"] = df["translated_lang"].apply(
    lambda x: x.lower())


In [8]:
df["translated_lang"].value_counts().head(20)

original sound                                   384
nan                                              217
original sound.                                   20
as original                                       19
therefore i am                                     8
original                                           6
oh no                                              5
the original sound                                 5
potential breakup song                             4
without you                                        4
where is the love?                                 4
what you know bout love                            3
they are original                                  3
pilfingerdansen - remix                            3
ruin our friendship                                3
sean kingston & justin bieber - eenie meenie       2
billie jean x boo x f it up vmeshbeats mashup      2
oh my god                                          2
sit on the farmer                             

In [9]:
df["original_sound"] = df["translated_lang"].apply(lambda x: True if "original" in  x else False)


In [10]:
df["first_audio"] = df["musicMeta.musicOriginal"]


In [11]:
df[["first_audio","original_sound", "likeRate"]].groupby(
    ["first_audio", "original_sound"]).mean().sort_values(by="likeRate", ascending=False)


likeRate
first_audio original_sound          
1           False           0.106613
            True            0.101524
0           False           0.087618
            True            0.075364

In [12]:
df["original_sound"].value_counts()

False    546
True     454
Name: original_sound, dtype: int64

In [15]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'text', 'createTime', 'webVideoUrl',
       'likeCount', 'shareCount', 'playCount', 'commentCount', 'downloaded',
       'mentions', 'hashtags', 'authorMeta.id', 'authorMeta.secUid',
       'authorMeta.name', 'authorMeta.nickName', 'authorMeta.verified',
       'authorMeta.signature', 'authorMeta.avatar', 'musicMeta.musicId',
       'musicMeta.musicName', 'musicMeta.musicAuthor',
       'musicMeta.musicOriginal', 'musicMeta.playUrl', 'musicMeta.coverThumb',
       'musicMeta.coverMedium', 'musicMeta.coverLarge', 'covers.default',
       'covers.origin', 'covers.dynamic', 'videoMeta.height',
       'videoMeta.width', 'videoMeta.duration', 'shareRate', 'likeRate',
       'commentRate', 'musicName', 'translated_lang', 'original_sound',
       'first_audio'],
      dtype='object')

In [36]:
filter_df = df[["first_audio","original_sound",'likeCount', 'shareCount', 'playCount', 'commentCount','shareRate','likeRate','commentRate']]
# filter_df['first_audio'] = filter_df['first_audio'].replace(1,'True').replace(0,'False')

count_df = filter_df.groupby(['first_audio','original_sound'],as_index=False).size().rename(columns={"size": "videos_count"})

# pivot_df = pivot_df[pivot_df['buyer'] == 1].reset_index().drop(columns=['index'])

pivot_df = filter_df.groupby(['first_audio','original_sound'],as_index=False)['likeCount', 'shareCount', 'playCount', 'commentCount'].agg([np.sum])
# pivot_df['share_per_vid'] = pivot_df[''] / pivot_df['videos_count'] # .sum() # round( * 100,2)
pivot_df = pivot_df.merge(count_df, on=['first_audio','original_sound'], how='left')
pivot_df
# pivot_df = pivot_df.groupby(["first_audio", "original_sound"])[]#.sort_values(by="likeRate", ascending=False)

/var/folders/fr/m4v_69gx29dghgljd9_f6r4w0000gn/T/ipykernel_12779/1033942570.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pivot_df = filter_df.groupby(['first_audio','original_sound'],as_index=False)['likeCount', 'shareCount', 'playCount', 'commentCount'].agg([np.sum])
/opt/anaconda3/envs/smm635_gp/lib/python3.8/site-packages/pandas/core/frame.py:9191: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


,first_audio,original_sound,"(likeCount, sum)","(shareCount, sum)","(playCount, sum)","(commentCount, sum)",videos_count
0,0,False,15170998,443832,211173736,176959,249
1,0,True,22246,103,284482,462,5
2,1,False,19415416,394291,238658547,145891,297
3,1,True,62189220,1067135,579096170,948223,449


In [23]:
count_df

,first_audio,original_sound,videos_count
0,False,False,249
1,False,True,5
2,True,False,297
3,True,True,449
